In [1]:
import pandas as pd
import folium
import json
from matplotlib import pyplot as plt
import numpy as np
import random
import ipywidgets
import seaborn as sns
from folium import plugins
import geojson
from geojson import LineString, Point, Feature, FeatureCollection, dump
from geopy.geocoders import Nominatim

pkgdir = "C:/Users/pietr/Documents/Arion/Trabalho VED/"

with open(f"{pkgdir}brazil-states.geojson", encoding='utf-8') as f:
    topo_estados = geojson.load(f)

for track in topo_estados['features']:
    with open(track['properties']['name'][0] + '.geojson', 'a+') as f:
        dump(track, f, indent=2)

In [2]:
geoestados = pd.read_csv(f"{pkgdir}estados.csv", index_col="nome")
segpub = pd.read_csv(f"{pkgdir}segurancapub.csv")
hom = pd.read_csv(f"{pkgdir}homicidiosbrasil.csv")
colors = sns.color_palette("hls", 9)

In [3]:
estados = []
for uf in segpub["UF"]:
    if not(uf in estados):
        estados.append(uf)

In [4]:
m = folium.Map(location=[-14.19, -48.79], zoom_start=4)

In [5]:
for uf in estados:
    folium.Marker(
        list(geoestados.loc[uf]["latitude":"longitude"]), popup=f"{uf}", tooltip="More Info",
        icon = folium.Icon(color="red", icon = "info-sign")
).add_to(m)

In [6]:
dfsanos = {}
for ano in segpub["Ano"]:
    if not(ano in dfsanos.keys()):
        dfsanos[ano] = segpub[segpub["Ano"] == ano]

dfscrimes_anos = {}
for ano in dfsanos:
    dfscrimes_anos[ano] = {}
    for crime in dfsanos[ano]["Tipo Crime"]:
        if not(crime in dfscrimes_anos[ano].keys()):
            dfscrimes_anos[ano][crime] = dfsanos[ano][dfsanos[ano]["Tipo Crime"] == crime]

In [34]:
data = {}
for ano in dfscrimes_anos:
    data[ano] = {}
    for crime in dfscrimes_anos[ano]:
        data[ano][crime] = {"UF":[],"Freq":[]}
        for uf in estados:
            data[ano][crime]["UF"].append(uf)
            data[ano][crime]["Freq"].append(sum(dfscrimes_anos[ano][crime][dfscrimes_anos[ano][crime]["UF"] == uf]["Ocorrências"]))

In [43]:
select_ano = ipywidgets.Select(
    options=list(dfscrimes_anos.keys()),
    value=2020,
    description="Ano",
    disabled=False
)
select_crime = ipywidgets.Select(
    options=list(dfscrimes_anos[2015].keys()),
    value="Estupro",
    description="Crime",
    disabled=False
)

def select(year, crime):
    for ano in data:
        if year == ano:
            m = folium.Map(location=[-14.19, -48.79], zoom_start=4)
            m.choropleth(geo_data = topo_estados,
                         name = "Dados",
                         data = pd.DataFrame(data[ano][crime]),
                         columns=["UF","Freq"],
                         key_on="feature.properties.name",
                         fill_color="Reds",
                         fill_opacity="0.7",
                         line_color="white",
                         line_opacity="0.8",
                         show=True,
                         legend_name="Nolose")
            
            for uf in estados:
                ocorr = sum(dfscrimes_anos[ano][crime][dfscrimes_anos[ano][crime]["UF"] == uf]["Ocorrências"])
                folium.Marker(
                    list(geoestados.loc[uf]["latitude":"longitude"]), popup=f"{uf}\n{ocorr}", tooltip="More Info",
                    icon = folium.Icon(color="red", icon = "info-sign")).add_to(m)
            plugins.Fullscreen(position='topleft').add_to(m)
            display(m)


ipywidgets.interact(select, year=select_ano, crime=select_crime)

interactive(children=(Select(description='Ano', index=1, options=(2021, 2020, 2019, 2018, 2017, 2016, 2015), v…

<function __main__.select(year, crime)>

In [25]:
sum(dfscrimes_anos[2018]["Estupro"][dfscrimes_anos[2018]["Estupro"]["UF"] == "Bahia"]["Ocorrências"])

3158